## Testing behaviour on concept drift

In [ ]:
# import the functions needed for validate and comparsion

import numpy as np
import cupy as cp
import pandas as pd
from timeit import default_timer as timer
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skmultiflow.data import FileStream, ConceptDriftStream, RandomRBFGenerator, RandomRBFGeneratorDrift
from skmultiflow.neural_networks import PerceptronMask
from skmultiflow.trees import HoeffdingTreeClassifier
from sklearn.neural_network import MLPClassifier


# using plotly for plots
#import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
# stability measurment as proposed in "Measurment the Stability of Feature Selection"
def pearson_stability_ij(arr1,arr2):
    d = len(arr1)
    k_i = np.sum(arr1)
    k_j = np.sum(arr2)

    # catch edge cases as proposed in the paper under 4.1
    if (k_i == 0 or k_i == d) and k_i != k_j :
        return 0
    elif (k_j == 0 or k_j == d) and k_i != k_j :
        return 0
    elif (k_i == 0 or k_i == d) and k_i == k_j :
        return 1
    x_hat_i = k_i / d
    x_hat_j = k_j / d
    arr1 = arr1 - x_hat_i
    arr2 = arr2 - x_hat_j
    dividend = 1/d * np.sum(arr1*arr2)
    divisor = np.sqrt(1/d*np.sum(arr1**2))*np.sqrt(1/d*np.sum(arr2**2))
    return dividend/divisor

def stability_factor(selected_ftrs):
   M = len(selected_ftrs)
   sum_stabilities = 0
   for i in range(M):
       for j in range(i+1, M):
           sum_stabilities += pearson_stability_ij(selected_ftrs[i], selected_ftrs[j])
   return 1/(M*(M-1))*sum_stabilities * 2     

In [ ]:
# import algorithms
from fires import FIRES
from ofs import MC_OFS
from ofssgr import MC_OFSSGD
from fsds import StreamFeatWeight

### Streaming Data Generation with Concept Drift


Extra Notebook for this data, cause it doesn't have test data and the stream is unlimited 

In [ ]:
# stream generator to test how algorithms perform on data with concept drift
stream = RandomRBFGeneratorDrift(model_random_state=99, sample_random_state = 50,
 n_classes = 6, n_features = 200, n_centroids = 50, change_speed=0.05,
 num_drift_centroids=25)

n_selected_ftr = 50
batch_size = 100
n_window = 10

In [ ]:
# stream 1
drift_stream = RandomRBFGenerator(model_random_state=42, sample_random_state=43, n_classes=10, n_features=200)
start_stream = RandomRBFGenerator(model_random_state=52, sample_random_state=53, n_classes=10, n_features=200)

stream = ConceptDriftStream(stream=start_stream, drift_stream=drift_stream, position=10000, width=10)

dataset_name = "Concept_drift_1"
n_selected_ftr = 50
n_window = 100
batch_size = 10

In [12]:
# stream 2
drift_stream = RandomRBFGenerator(model_random_state=42, sample_random_state=43, n_classes=10, n_features=200)
start_stream = RandomRBFGenerator(model_random_state=52, sample_random_state=53, n_classes=10, n_features=200)

stream = ConceptDriftStream(stream=start_stream, drift_stream=drift_stream, position=10000, width=1000)

dataset_name = "Concept_drift_2"
n_selected_ftr = 50
n_window = 100
batch_size = 10

In [13]:
# prepare folder for plots
folder = "plots/concept_drift/{}".format(dataset_name)
if not os.path.exists(folder):
    os.makedirs(folder)

export_type = "pdf" # "png", "jpeg", "webp", "pdf", "svg"

### FIRES

In [14]:
stream.restart()
predictor = MLPClassifier()
x,y = stream.next_sample(batch_size=batch_size)
predictor.partial_fit(x,y, stream.target_values)

MLPClassifier()

In [15]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=stream.target_values,
                    mu_init=0,
                    sigma_init=1,
                    model='softmax',
                    lr_mu=0.1,
                    lr_sigma=0.1)        

In [16]:
fires_accuracy = []
fires_f1 = []


fires_selected_ftrs = []
fires_stability = []

start_time_all = timer()
# stream has unlimeted samples so set for loop
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=batch_size)
    # Select features
    
    ftr_weights = fires_model.weigh_features(x,y)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fires_selected_ftrs.append(ftr_array)

    if len(fires_selected_ftrs) >= 10:
        stability = stability_factor(fires_selected_ftrs[-10:])
        fires_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    fires_accuracy.append(accuracy_score(y, y_pred))
    fires_f1.append(f1_score(y, y_pred, average="weighted", labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
fires_run_time = timer() - start_time_all
print("The whole FIRES run took {}".format(fires_run_time))

fires_moving_average = pd.Series(fires_accuracy).rolling(window=n_window).mean().iloc[n_window-1:].values
fires_f1 = pd.Series(fires_f1).rolling(window=n_window).mean().iloc[n_window-1:].values

rol this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and 

### FSDS

In [17]:
stream.restart()
predictor = MLPClassifier()
x,y = stream.next_sample(batch_size=batch_size)
predictor.partial_fit(x,y, stream.target_values)

MLPClassifier()

In [18]:
fsds_model = StreamFeatWeight(m=stream.n_features, k=stream.n_classes)
fsds_model.low_rank_approximation(x.T) # needs some pretraining in the first run

/home/kitten/BA_FIRES/fires/fsds.py:71: RuntimeWarning: invalid value encountered in true_divide
  D = np.diag(s[:self.k] / (s[:self.k] ** 2 + alpha))


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [19]:
fsds_accuracy = []
fsds_f1 = []


fsds_selected_ftrs = []
fsds_stability = []

start_time_all = timer()
# stream has unlimeted samples so set for loop
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=batch_size)
    # Select features
    
    ftr_weights = fsds_model.low_rank_approximation(x.T)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fsds_selected_ftrs.append(ftr_array)

    if len(fsds_selected_ftrs) >= 10:
        stability = stability_factor(fsds_selected_ftrs[-10:])
        fsds_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    fsds_accuracy.append(accuracy_score(y, y_pred))
    fsds_f1.append(f1_score(y, y_pred, average="weighted", labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
fsds_run_time = timer() - start_time_all
print("The whole fsds run took {}".format(fsds_run_time))

fsds_moving_average = pd.Series(fsds_accuracy).rolling(window=n_window).mean().iloc[n_window-1:].values
fsds_f1 = pd.Series(fsds_f1).rolling(window=n_window).mean().iloc[n_window-1:].values

trol this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and

In [20]:
ftr_weights

array([0.01255527, 0.01989479, 0.01137272, 0.02022036, 0.0215047 ,
       0.01461718, 0.01117053, 0.01488202, 0.01788044, 0.01358257,
       0.01150563, 0.01351954, 0.01019825, 0.0159516 , 0.01495969,
       0.01268442, 0.01781789, 0.01059123, 0.01699045, 0.01541616,
       0.01275202, 0.01287889, 0.01255511, 0.0208066 , 0.01349193,
       0.01224399, 0.01024313, 0.01524544, 0.01601379, 0.01597838,
       0.0122225 , 0.03029542, 0.01747142, 0.01348406, 0.01812207,
       0.01952962, 0.02267922, 0.01208119, 0.0128183 , 0.01867492,
       0.01637137, 0.01534   , 0.01704515, 0.01454128, 0.0146258 ,
       0.01628131, 0.00871533, 0.01789573, 0.01348507, 0.01225616,
       0.01213923, 0.02029407, 0.0151809 , 0.01803046, 0.01319252,
       0.01578938, 0.01690009, 0.01390825, 0.01712649, 0.01696752,
       0.01425223, 0.01877329, 0.01293111, 0.01493803, 0.01555521,
       0.02046072, 0.01428637, 0.01771793, 0.01438387, 0.01098379,
       0.01258619, 0.00987728, 0.01066233, 0.0143025 , 0.01567

### OFS

In [21]:
stream.restart()
predictor = MLPClassifier()
x,y = stream.next_sample(batch_size=batch_size)
predictor.partial_fit(x,y, stream.target_values)

MLPClassifier()

In [22]:
ofs_model = MC_OFS(regularization_param = 0.01, step_size = 0.2, n_selected_ftr=n_selected_ftr, n_total_ftr=stream.n_num_features, n_classes=stream.n_classes)

In [23]:
ofs_accuracy = []
ofs_f1 = []


ofs_selected_ftrs = []
ofs_stability = []

start_time_all = timer()
# stream has unlimeted samples so set for loop
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=batch_size)
    # Select features

     # Select features
    for idx, label in enumerate(np.array(y, dtype='int')):
        ofs_model.train(x[idx],label)
    
    
    ftr_selection = ofs_model.get_feature_indices()

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    ofs_selected_ftrs.append(ftr_array)

    if len(ofs_selected_ftrs) >= 10:
        stability = stability_factor(ofs_selected_ftrs[-10:])
        ofs_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    ofs_accuracy.append(accuracy_score(y, y_pred))
    ofs_f1.append(f1_score(y, y_pred, average="weighted", labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
ofs_run_time = timer() - start_time_all
print("The whole ofs run took {}".format(ofs_run_time))

ofs_moving_average = pd.Series(ofs_accuracy).rolling(window=n_window).mean().iloc[n_window-1:].values
ofs_f1 = pd.Series(ofs_f1).rolling(window=n_window).mean().iloc[n_window-1:].values

trol this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and

### OFSSGD

In [24]:
stream.restart()
predictor = MLPClassifier()
x,y = stream.next_sample(batch_size=batch_size)
predictor.partial_fit(x,y, stream.target_values)

MLPClassifier()

In [25]:
ofssgd_model = MC_OFSSGD(reduction_threshold=0.4, reduction_value=0.1, regularization_param=0.01, step_size=0.2, n_total_ftrs=stream.n_num_features, n_classes=stream.n_classes)

In [26]:
ofssgd_accuracy = []
ofssgd_f1 = []


ofssgd_selected_ftrs = []
ofssgd_stability = []

start_time_all = timer()
# stream has unlimeted samples so set for loop
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=batch_size)
    # Select features
    
    for idx, label in enumerate(np.array(y, dtype='int')):
        ofssgd_model.train(x[idx],label)

    ftr_selection = ofssgd_model.get_feature_indices()

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    ofssgd_selected_ftrs.append(ftr_array)

    if len(ofssgd_selected_ftrs) >= 10:
        stability = stability_factor(ofssgd_selected_ftrs[-10:])
        ofssgd_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    ofssgd_accuracy.append(accuracy_score(y, y_pred))
    ofssgd_f1.append(f1_score(y, y_pred, average="weighted", labels=stream.target_values))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
end_time_all = timer()
ofssgd_run_time = timer() - start_time_all
print("The whole ofssgd run took {}".format(ofssgd_run_time))

ofssgd_moving_average = pd.Series(ofssgd_accuracy).rolling(window=n_window).mean().iloc[n_window-1:].values
ofssgd_f1 = pd.Series(ofssgd_f1).rolling(window=n_window).mean().iloc[n_window-1:].values

ol this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and b

In [27]:
# stability
title = "Stability"
col_names = ["FIRES", "OFS", "OFSSGD", "FSDS"]
d = {"FIRES":fires_stability, "OFS":ofs_stability, "OFSSGD":ofssgd_stability, "FSDS":fsds_stability} #"random":random_stability 'FSDS': 'green',
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y = col_names, title=title, labels={"index":"batches", "value":"stability"},color_discrete_map={'FIRES': 'red', 
                                                    'OFS': 'purple', "OFSSGD":"yellow",'FSDS': 'green'} )
stability_trace = fig['data']
fig.write_image("{}/stability.{}".format(folder, export_type))
fig.show()

In [28]:
#moving averages
title = "Moving averages over accuracy while learning with window {}".format(n_window)
col_names = ["FIRES", "OFS", "OFSSGD", "FSDS"]
d = {"FIRES":fires_moving_average, "OFS":ofs_moving_average, 
"OFSSGD":ofssgd_moving_average, "FSDS":fsds_moving_average}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y=col_names, title=title, labels={"index":"batches", "value":"accuracy"}, color_discrete_map={'FIRES': 'red', 
                                                    'OFS': 'purple', "OFSSGD":"yellow",'FSDS': 'green'})
accuracy_trace = fig['data']
fig.write_image("{}/accuracy.{}".format(folder, export_type))
fig.show()


In [29]:
#moving averages
title = "Moving averages over f1 while learning with window {}".format(n_window)
col_names = ["FIRES", "OFS", "OFSSGD", "FSDS"]
d = {"FIRES":fires_f1, "OFS":ofs_f1, 
"OFSSGD":ofssgd_f1, "FSDS":fsds_f1}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y=col_names, title=title, labels={"index":"batches", "value":"accuracy"}, color_discrete_map={'FIRES': 'red', 
                                                    'OFS': 'purple', "OFSSGD":"yellow", "random":"cyan",'FSDS': 'green'})
f1_trace = fig['data']
fig.write_image("{}/f1.{}".format(folder, export_type))
fig.show()




In [30]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1) # subplot_titles=("Stability","Accuracy", "F1-Score"),
for i in range(len(stability_trace)):
    stability_trace[i]["showlegend"] = False
    fig.add_trace(stability_trace[i], row=1, col=1)
for i in range(len(accuracy_trace)):
    fig.add_trace(accuracy_trace[i], row=2, col=1)
for i in range(len(f1_trace)):
    f1_trace[i]["showlegend"] = False
    fig.add_trace(f1_trace[i], row=3, col=1)
fig.update_xaxes(title_text="batches", row=3, col=1)
fig.update_yaxes(title_text="Stability", row=1, col=1)
fig.update_yaxes(title_text="Accuracy", row=2, col=1)
fig.update_yaxes(title_text="F1-Score", row=3, col=1)
fig.write_image("{}/all_scores.{}".format(folder, export_type))

fig.show()